In [1]:
%%time
#To hide warnings export PYTHONWARNINGS="ignore"
#Imports{

import os
from os.path import dirname
from os.path import join
os.environ['KERAS_BACKEND'] = 'theano'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Cha


import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import cPickle as pickle
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.misc import imread
from scipy.misc import imresize
from scipy.io import loadmat
from scipy.interpolate import spline
from sklearn.cluster import MiniBatchKMeans

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.cluster import KMeans

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D, Input, ZeroPadding2D,merge,Lambda,Conv2D
from keras.layers.convolutional import Convolution2D
from keras.models import Model
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.optimizers import SGD
from keras import backend as K
from keras.engine import Layer
from keras.layers.core import Lambda
from keras.utils.vis_utils import plot_model
from keras.layers.core import  Lambda
from keras.regularizers import l2
import cv2
#}

Using Theano backend.


CPU times: user 632 ms, sys: 221 ms, total: 854 ms
Wall time: 835 ms


In [2]:
#Code snippet needed to read activation values from each layer of the pre-trained artificial neural networks
def get_activations(model, layer, X_batch):
    #keras.backend.function(inputs, outputs, updates=None)
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output,])
    #The learning phase flag is a bool tensor (0 = test, 1 = train)
    activations = get_activations([X_batch,0])
    return activations

In [3]:
def preprocess_image(image_paths, image_height=224, image_width=224,color_mode='rgb'):
    """resize images to the appropriate dimensions
    :param image_width:
    :param image_height:
    :param image: image
    :return: image
    """
    img_list = []
    
    for im_path in image_paths:
        image = imread(im_path, mode='RGB')
        image = cv2.resize(image, (image_height, image_width))
    
        if color_mode == 'bgr':
            image = image.transpose((2, 0, 1))
        img_list.append(image)
        
    try:
        img_batch = np.stack(img_list, axis=0)
    except:
        print(im_path)
        raise ValueError('when img_size and crop_size are None, images'
                ' in image_paths must have the same shapes.')
    return img_batch

In [4]:
#Helper function to normalization across channels
K.set_image_dim_ordering('th')
def crosschannelnormalization(alpha=1e-4, k=2, beta=0.75, n=5, **kwargs):
    """
    This is the function used for cross channel normalization in the original
    Alexnet
    """
    def f(X):
        if K.image_dim_ordering()=='tf':
            b, r, c, ch = X.get_shape()
        else:
            b, ch, r, c = X.shape

        half = n // 2
        square = K.square(X)
        scale = k
        if K.image_dim_ordering() == 'th':
            extra_channels = K.spatial_2d_padding(K.permute_dimensions(square, (0, 2, 3, 1)), ((0,0),(half,half)))
            extra_channels = K.permute_dimensions(extra_channels, (0, 3, 1, 2))
            for i in range(n):
                scale += alpha * extra_channels[:, i:i+ch, :, :]
        if K.image_dim_ordering() == 'tf':
            extra_channels = K.spatial_2d_padding(K.permute_dimensions(square, (0, 3, 1, 2)), (half, 0))
            extra_channels = K.permute_dimensions(extra_channels, (0, 2, 3, 1))
            for i in range(n):
                scale += alpha * extra_channels[:, :, :, i:i+int(ch)]
        scale = scale ** beta
        return X / scale


    return Lambda(f, output_shape=lambda input_shape: input_shape, **kwargs)

In [5]:
#Helper Function to split tensor
def splittensor(axis=1, ratio_split=1, id_split=0, **kwargs):
    def f(X):
        div = K.shape(X)[axis] // ratio_split

        if axis == 0:
            output = X[id_split*div:(id_split+1)*div, :, :, :]
        elif axis == 1:
            output = X[:, id_split*div:(id_split+1)*div, :, :]
        elif axis == 2:
            output = X[:, :, id_split*div:(id_split+1)*div, :]
        elif axis == 3:
            output = X[:, :, :, id_split*div:(id_split+1)*div]
        else:
            raise ValueError("This axis is not possible")
        return output

    def g(input_shape):
        output_shape = list(input_shape)
        output_shape[axis] = output_shape[axis] // ratio_split
        return tuple(output_shape)


    return Lambda(f, output_shape=lambda input_shape: g(input_shape), **kwargs)

In [6]:
#Alexnet layer architecture class
def AlexNet(img_shape=(3, 227, 227), n_classes=1000, l2_reg=0.,weights_path=None, lambda_mask=None):

    dim_ordering = K.image_dim_ordering()
    print dim_ordering
    if dim_ordering == 'th':
        batch_index = 0
        channel_index = 1
        row_index = 2
        col_index = 3
    if dim_ordering == 'tf':
        batch_index = 0
        channel_index = 3
        row_index = 1
        col_index = 2
        
    
    inputs = Input(img_shape)

    conv_1 = Convolution2D(96, 11, 11, subsample=(4, 4), activation='relu',
                           name='conv_1', W_regularizer=l2(l2_reg))(inputs)

    if lambda_mask is not None:
        conv_1_mask  = np.reshape(lambda_mask[0:290400], (96,55,55))
    else:
        conv_1_mask = np.ones(shape=((96, 55, 55)))
    
    conv_1_mask  = K.variable(conv_1_mask)
    conv_1_lambda = Lambda(lambda x: x * conv_1_mask)(conv_1)

    conv_2 = MaxPooling2D((3, 3), strides=(2, 2))(conv_1_lambda)
    conv_2 = crosschannelnormalization(name="convpool_1")(conv_2)
    conv_2 = ZeroPadding2D((2, 2))(conv_2)
    conv_2 = merge([
        Convolution2D(128, 5, 5, activation="relu", name='conv_2_'+str(i+1),
                      W_regularizer=l2(l2_reg))(
            splittensor(axis=channel_index, ratio_split=2, id_split=i)(conv_2)
        ) for i in range(2)], mode='concat', concat_axis=channel_index, name="conv_2")

    if lambda_mask is not None:
        conv_2_mask  = np.reshape(lambda_mask[290400:477024],(256, 27, 27) )
    else:
        conv_2_mask = np.ones(shape=((256, 27, 27)))
        
    conv_2_mask = K.variable(conv_2_mask)
    conv_2_lambda = Lambda(lambda x: x * conv_2_mask)(conv_2)

    conv_3 = MaxPooling2D((3, 3), strides=(2, 2))(conv_2_lambda)
    conv_3 = crosschannelnormalization()(conv_3)
    conv_3 = ZeroPadding2D((1, 1))(conv_3)
    conv_3 = Convolution2D(384, 3, 3, activation='relu', name='conv_3',
                           W_regularizer=l2(l2_reg))(conv_3)

    if lambda_mask is not None:
        conv_3_mask  = np.reshape(lambda_mask[477024:541920],(384, 13, 13))
    else:
        conv_3_mask = np.ones(shape=((384, 13, 13)))
    
    conv_3_mask = K.variable(conv_3_mask)
    conv_3_lambda = Lambda(lambda x: x * conv_3_mask)(conv_3)

    conv_4 = ZeroPadding2D((1, 1))(conv_3_lambda)
    conv_4 = merge([
        Convolution2D(192, 3, 3, activation="relu", name='conv_4_'+str(i+1),
                      W_regularizer=l2(l2_reg))(
            splittensor(axis=channel_index, ratio_split=2, id_split=i)(conv_4)
        ) for i in range(2)], mode='concat', concat_axis=channel_index, name="conv_4")

    if lambda_mask is not None:
        conv_4_mask  = np.reshape(lambda_mask[541920:606816],(384, 13, 13))
    else:
        conv_4_mask = np.ones(shape=((384, 13, 13)))
        
    conv_4_mask = K.variable(conv_4_mask)
    conv_4_lambda = Lambda(lambda x: x * conv_4_mask)(conv_4)

    conv_5 = ZeroPadding2D((1, 1))(conv_4_lambda)
    conv_5 = merge([
        Convolution2D(128, 3, 3, activation="relu", name='conv_5_'+str(i+1),
                      W_regularizer=l2(l2_reg))(
            splittensor(axis=channel_index, ratio_split=2, id_split=i)(conv_5)
        ) for i in range(2)], mode='concat', concat_axis=channel_index, name="conv_5")

    if lambda_mask is not None:
        conv_5_mask  = np.reshape(lambda_mask[606816:650080],(256, 13, 13))
    else:
        conv_5_mask = np.ones(shape=((256, 13, 13)))
    
    conv_5_mask = K.variable(conv_5_mask)
    conv_5_lambda = Lambda(lambda x: x * conv_5_mask)(conv_5)

    dense_1 = MaxPooling2D((3, 3), strides=(2, 2), name="convpool_5")(conv_5_lambda)

    dense_1 = Flatten(name="flatten")(dense_1)
    dense_1 = Dense(4096, activation='relu', name='dense_1',
                    W_regularizer=l2(l2_reg))(dense_1)

    if lambda_mask is not None:
        dense_1_mask  = np.reshape(lambda_mask[650080:654176],(4096,))
    else:
        dense_1_mask = np.ones(shape=((4096,)))
    
    
    dense_1_mask = K.variable(dense_1_mask)
    dense_1_lambda = Lambda(lambda x: x * dense_1_mask)(dense_1)

    dense_2 = Dropout(0.5)(dense_1_lambda)
    dense_2 = Dense(4096, activation='relu', name='dense_2',
                    W_regularizer=l2(l2_reg))(dense_2)

    if lambda_mask is not None:
        dense_2_mask  = np.reshape(lambda_mask[654176:658272],(4096,))
    else:
        dense_2_mask = np.ones(shape=((4096,)))
    
    dense_2_mask = K.variable(dense_2_mask)
    dense_2_lambda = Lambda(lambda x: x * dense_2_mask)(dense_2)

    dense_3 = Dropout(0.5)(dense_2_lambda)
    if n_classes == 1000:
        dense_3 = Dense(n_classes, name='dense_3',
                        W_regularizer=l2(l2_reg))(dense_3)
    else:
        # We change the name so when loading the weights_file from a
        # Imagenet pretrained model does not crash
        dense_3 = Dense(n_classes, name='dense_3_new',
                        W_regularizer=l2(l2_reg))(dense_3)


    prediction = Activation("softmax", name="softmax")(dense_3)

    model = Model(input=inputs, output=prediction)
    if weights_path:
        model.load_weights(weights_path)
    
    return model

In [7]:
#Load the details of all the 1000 classes and the function to conver the synset id to words{
meta_clsloc_file = '../../data/meta_clsloc.mat'
synsets = loadmat(meta_clsloc_file)['synsets'][0]
synsets_imagenet_sorted = sorted([(int(s[0]), str(s[1][0])) for s in synsets[:1000]],key=lambda v: v[1])
corr = {}
for j in range(1000):
    corr[synsets_imagenet_sorted[j][0]] = j

corr_inv = {}
for j in range(1, 1001):
    corr_inv[corr[j]] = j

def id_to_words(id_):
    return synsets[corr_inv[id_] - 1][2][0]

def pprint_output(out, n_max_synsets=10):
    wids = []
    best_ids = out.argsort()[::-1][:10]
    for u in best_ids:
        wids.append(str(synsets[corr_inv[u] - 1][1][0]))
    #print('%.2f' % round(100 * out[u], 2) + ' : ' + id_to_words(u)+' '+ str(synsets[corr_inv[u] - 1][1][0]))
    return wids


In [8]:
#Code snippet to load the ground truth labels to measure the performance{
truth = {}
with open('../../data/ILSVRC2014_clsloc_validation_ground_truth.txt') as f:
    line_num = 1
    for line in f.readlines():
        ind_ = int(line)
        temp  = None
        for i in synsets_imagenet_sorted:
            if i[0] == ind_:
                temp = i
        #print ind_,temp
        if temp != None:
            truth[line_num] = temp
        else:
            print '##########', ind_
            pass
        line_num += 1
#}

In [9]:
#Function to predict the top 5 accuracy
def top5accuracy(true, predicted):
    assert len(true) == len(predicted)
    result = []
    flag  = 0
    for i in range(len(true)):
        flag  = 0
        temp = true[i]
        for j in predicted[i]:
            if j == temp:
                flag = 1
                break
        if flag == 1:
            result.append(1)
        else:
            result.append(0)
    counter = 0.
    for i in result:
        if i == 1:
            counter += 1.
    error = 1.0 - counter/float(len(result))
    #print len(np.where(np.asarray(result) == 1)[0])
    return len(np.where(np.asarray(result) == 1)[0]), error


In [ ]:
%%time
#Testing on test data{
data_path = '../../data/pkl/'
classes = ['animate','inanimate']

result = {}

with open(data_path+classes[0]+'_test.pkl') as f:
    X_fold = pickle.load(f)
with open(data_path+classes[1]+'_test.pkl') as f:
    y_fold = pickle.load(f)

X = np.column_stack((X_fold,y_fold))  
if os.path.exists('../../data/pkl/kmeans_first_test.pickle'):
    with open('../../data/pkl/kmeans_first_test.pickle',"rb") as f:
        X_new,pred_kmeans,kmeans = pickle.load(f)
else:   
   
    kmeans = MiniBatchKMeans(n_clusters=65827,
                             random_state=0,
                             batch_size=6,
                             max_iter=10).fit(X)
    #print kmeans.cluster_centers_
    pred_kmeans = kmeans.predict(X)
    X_new = kmeans.cluster_centers_


#DO CLUSTERING AND GET CLUSTERS

from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import Birch
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
import genieclust
import hdbscan
import smm

j = 22 #Set this value from scree plot!
method = 'GMM'
print j
#clf = hdbscan.HDBSCAN(min_cluster_size=j, gen_min_span_tree=True)
#clf = DBSCAN(eps=5.443)
#clf = KMeans(n_clusters=j,random_state=143)
#clf= SpectralClustering(n_clusters=j,random_state=143)
#clf =  AgglomerativeClustering(n_clusters=j, linkage='ward')
#clf = Birch(branching_factor=50, n_clusters=j, threshold=0.5,compute_labels=True)
clf = GaussianMixture(n_components=j, covariance_type='full',random_state=143)
#clf= genieclust.genie.Genie(n_clusters=j)
#clf= smm.SMM(n_components=j, covariance_type='full', random_state=143, tol=1e-12,min_covar=1e-6, n_iter=1000, n_init=1, params='wmcd', init_params='wmcd')
temp = clf.fit(X_new)
y_pred = clf.predict(X_new)
#y_pred = clf.fit_predict(X_new)
print set(y_pred)
#Z = clf.predict(X)

for label in set(y_pred):
    print('Cluster: ',j,'Label: ', label)

    #Lesioning and measuring performance
    #pred = clf.fit_predict(X_new)
    temp = clf.fit(X_new)
    pred = clf.predict(X_new)
    loc = np.where(pred==label)
    loc_temp = kmeans.predict(X_new[loc[0]])
    loc_new =[]
    for entry in set(loc_temp):
        temp = np.where(pred_kmeans==entry)[0]
        loc_new.extend(temp)

    lambda_mask = np.ones(shape=((658272,)))
    lambda_mask[loc_new] = 0.

    #plt.scatter(X[:,0],X[:,1], c=y_pred) 

    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model = AlexNet(weights_path="../../data/weights/alexnet_weights.h5",lambda_mask=lambda_mask)
    model.compile(optimizer=sgd, loss='mse')

    flag = 0
    dprime = 0.
    for p in classes:
        im_valid_test = []
        image_list_valid = '../../data/pkl/'+p+'_image_list_test.txt'
        with open(image_list_valid,'rb') as f:
            for line in f.readlines():
                im_valid_test.append(line.strip('\n'))
        im_temp = preprocess_image(im_valid_test,227,227, color_mode="bgr")
        out = model.predict(im_temp,batch_size=64)
        
        true_valid_wids = []
        for i in im_valid_test:
                temp1 = i.split('/')[4]
                temp = temp1.split('.')[0].split('_')[2]
                true_valid_wids.append(truth[int(temp)][1])

        predicted_valid_wids = []
        for i in range(len(im_valid_test)):
            #print im_list[i], pprint_output(out[i]), true_wids[i]
            predicted_valid_wids.append(pprint_output(out[i]))

        count, error  = top5accuracy(true_valid_wids, predicted_valid_wids)

        print str(p)+' '+str(count)+' '+str(len(im_valid_test))+' '+str(error)+' '+str(1-error)

        if flag == 0:
            dprime = error
            flag = 1
        else:
            dprime -= error

    result[label] = dprime

In [ ]:
print(label)

In [ ]:
result.values(),X.shape

In [ ]:
z_temp = []
for item in y_pred:
    z_temp.append(result[item])
print(len(z_temp),len(X_new))
loc_z = kmeans.predict(X_new)
z = np.ones(shape=((658272,)))
for i in range(len(loc_z)):
    temp = np.where(pred_kmeans==loc_z[i])[0]
    z[temp] = z_temp[i]

In [ ]:
X.shape

In [ ]:
x = X[:,0]
y = X[:,1]

In [ ]:
#Density Plot for Animate/Inanimate

print(x.shape,y.shape,z.shape)
fig, ax = plt.subplots()
cs = ax.scatter(x, y, c=z, s=10,cmap='coolwarm')
cbar = fig.colorbar(cs)
lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]
ax.plot(lims, lims, 'k-', alpha=0.75)
plt.xlabel('Animate')
plt.ylabel('Inanimate')
plt.title('Performance Impact - Density plot')
plt.savefig('../../results/scree/'+str(method)+'_results_density.png', format='png',dpi=200)

In [ ]:
print result.values().index(max(result.values())), result.values().index(min(result.values()))
ana = int(result.values().index(max(result.values())))
ina = int(result.values().index(min(result.values())))
print result[ana], -1*(result[ina])

In [ ]:
fig, ax = plt.subplots()
label_loc = np.where(pred==ana)[0]
#print len(label_loc)
Zx = []
Zy = []
for i in range(len(label_loc)):
    temp = np.where(pred_kmeans==label_loc[i])[0]
    for i in temp:
        Zx.append(X[i][0])
        Zy.append(X[i][1])
#print len(np.where(Z!=0)[0])
#print Z
sc = plt.scatter(Zx,Zy) 
plt.xlim([-2.2,300])
plt.ylim([-2.2,300])
lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'k-', alpha=0.75, zorder=1)
plt.xlabel('Animate')
plt.ylabel('Inanimate')
plt.title('Most selective cluster for animate class')
plt.savefig('../../results/scree/'+str(method)+'_results_ana.png', format='png')

In [ ]:
fig, ax = plt.subplots()
label_loc = np.where(pred==ina)[0]
print len(label_loc)
Zx = []
Zy = []
for i in range(len(label_loc)):
    temp = np.where(pred_kmeans==label_loc[i])[0]
    for i in temp:
        Zx.append(X[i][0])
        Zy.append(X[i][1])
sc = plt.scatter(Zx,Zy) 
plt.xlim([-2.2,300])
plt.ylim([-2.2,300])
lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'k-', alpha=0.75, zorder=1)
plt.xlabel('Animate')
plt.ylabel('Inanimate')
plt.title('Most selective cluster for inanimate class')
plt.savefig('../../results/scree/'+str(method)+'_results_ina.png', format='png')

In [ ]:
label_loc = np.where(pred==ana)[0]
Z = []
for i in range(len(label_loc)):
    temp = np.where(pred_kmeans==label_loc[i])[0]
    for i in temp:
        Z.append(i)
print(max(Z))

In [ ]:
label_loc = np.where(pred==ana)[0]
Z = []
for i in range(len(label_loc)):
    temp = np.where(pred_kmeans==label_loc[i])[0]
    for i in temp:
        Z.append(i)
X = np.arange(7)
Y = np.zeros((7,))

for i in Z:
    if i in range(0,290400): 
        Y[0] += 1
    elif i in range(290400,477024):
        Y[1] += 1
    elif i in range(477024,541920):
        Y[2] += 1
    elif i in range(541920,606816):
        Y[3] += 1
    elif i in range(606816,650080):
        Y[4] += 1
    elif i in range(650080,654176):
        Y[5] += 1
    elif i in range(654176,658272):
        Y[6] += 1
    else:
        print i

Y[0] = float(Y[0]) /290400
Y[1] = float(Y[0]) / 196624
Y[2] = float(Y[2]) / 64896
Y[3] = float(Y[3]) / 64896
Y[4] = float(Y[4]) /43264
Y[5] = float(Y[5]) /4096
Y[6] = float(Y[6]) /4096

plt.ylim([0,1.])
rect = plt.bar(X,Y)
plt.xticks(X, ('conv_1', 'conv_2', 'conv_3', 'conv_4','conv_5','fc1','fc2'))
plt.ylabel('Relative count of neurons')
plt.title('Neurons from the animate cluster')
#autolabel(rect)
plt.savefig('../../results/scree/'+str(method)+'_results_ana_hist_alt.png', format='png')

In [ ]:
label_loc = np.where(pred==ina)[0]
Z = []
for i in range(len(label_loc)):
    temp = np.where(pred_kmeans==label_loc[i])[0]
    for i in temp:
        Z.append(i)
X = np.arange(7)
Y = np.zeros((7,))

for i in Z:
    if i in range(0,290400): 
        Y[0] += 1
    elif i in range(290400,477024):
        Y[1] += 1
    elif i in range(477024,541920):
        Y[2] += 1
    elif i in range(541920,606816):
        Y[3] += 1
    elif i in range(606816,650080):
        Y[4] += 1
    elif i in range(650080,654176):
        Y[5] += 1
    elif i in range(654176,658272):
        Y[6] += 1
    else:
        print i

Y[0] = float(Y[0]) /290400
Y[1] = float(Y[0]) / 196624
Y[2] = float(Y[2]) / 64896
Y[3] = float(Y[3]) / 64896
Y[4] = float(Y[4]) /43264
Y[5] = float(Y[5]) /4096
Y[6] = float(Y[6]) /4096

plt.ylim([0,1.])
rect = plt.bar(X,Y)
plt.ylabel('Relative count of neurons')
plt.title('Neurons from the inanimate cluster')
plt.xticks(X, ('conv_1', 'conv_2', 'conv_3', 'conv_4','conv_5','fc1','fc2'))
#autolabel(rect)
plt.savefig('../../results/scree/'+str(method)+'_results_ina_hist_alt.png', format='png')

In [ ]:
layer_info = {}
#Comparing Layer lesions
classes = ['animate','inanimate']
%time
for label in [ana,ina]:
    layer_info[label] = {}
    for layer in [1,2,3,4,5,6,7]:
        if layer == 1:
            start = 0
            end = 290400
        elif layer == 2:
            start = 290400
            end = 477024
        elif layer == 3:
            start = 477024
            end = 541920
        elif layer == 4:
            start = 541920
            end = 606816
        elif layer == 5:
            start = 606816
            end = 650080
        elif layer == 6:
            start = 650080
            end = 654176
        elif layer == 7:
            start = 654177
            end = 658272
    
        layer_info[label][layer] = {}
    
        #No lesion
        #print('No-lesioning')
        #print('Label:',label)
        #print('Layer:',layer)

        pred = clf.predict(X_new)
        lambda_mask = np.ones(shape=((658272,)))
       

        sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
        model = AlexNet(weights_path="../../data/weights/alexnet_weights.h5",lambda_mask=lambda_mask)
        model.compile(optimizer=sgd, loss='mse')

        flag = 0
        dprime = 0.
        for p in classes:
            im_valid_test = []
            image_list_valid = '../../data/pkl/'+p+'_image_list_test.txt'
            with open(image_list_valid,'rb') as f:
                for line in f.readlines():
                    im_valid_test.append(line.strip('\n'))
            im_temp = preprocess_image(im_valid_test,227,227, color_mode="bgr")
            out = model.predict(im_temp,batch_size=64)

            true_valid_wids = []
            for i in im_valid_test:
                    temp1 = i.split('/')[4]
                    temp = temp1.split('.')[0].split('_')[2]
                    true_valid_wids.append(truth[int(temp)][1])

            predicted_valid_wids = []
            for i in range(len(im_valid_test)):
                #print im_list[i], pprint_output(out[i]), true_wids[i]
                predicted_valid_wids.append(pprint_output(out[i]))

            count, error  = top5accuracy(true_valid_wids, predicted_valid_wids)

            print str(p)+' '+str(count)+' '+str(len(im_valid_test))+' '+str(error)+' '+str(1-error)
              
            if flag == 0:
                dprime = error
                flag = 1
            else:
                dprime -= error
        print('Layer: ',layer,'Label: ', label)
        print('No lesion: ',dprime)
        layer_info[label][layer]['no'] = dprime
        
        
        #Before lesion
        #print('Pre-layer-lesioning')
        #print('Label:',label)
        #print('Layer:',layer)

        pred = clf.predict(X_new)
        loc = np.where(pred==label)[0]
        loc_new =[]
        for i in range(len(loc)):
            temp = np.where(pred_kmeans==loc[i])[0]
            loc_new.extend(temp)

        lambda_mask = np.ones(shape=((658272,)))
        lambda_mask[loc_new] = 0.
        print('pre-loc', len(loc_new))
        sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
        model = AlexNet(weights_path="../../data/weights/alexnet_weights.h5",lambda_mask=lambda_mask)
        model.compile(optimizer=sgd, loss='mse')

        flag = 0
        dprime = 0.
        for p in classes:
            im_valid_test = []
            image_list_valid = '../../data/pkl/'+p+'_image_list_test.txt'
            with open(image_list_valid,'rb') as f:
                for line in f.readlines():
                    im_valid_test.append(line.strip('\n'))
            im_temp = preprocess_image(im_valid_test,227,227, color_mode="bgr")
            out = model.predict(im_temp,batch_size=64)

            true_valid_wids = []
            for i in im_valid_test:
                    temp1 = i.split('/')[4]
                    temp = temp1.split('.')[0].split('_')[2]
                    true_valid_wids.append(truth[int(temp)][1])

            predicted_valid_wids = []
            for i in range(len(im_valid_test)):
                #print im_list[i], pprint_output(out[i]), true_wids[i]
                predicted_valid_wids.append(pprint_output(out[i]))

            count, error  = top5accuracy(true_valid_wids, predicted_valid_wids)

            print str(p)+' '+str(count)+' '+str(len(im_valid_test))+' '+str(error)+' '+str(1-error)

            if flag == 0:
                dprime = error
                flag = 1
            else:
                dprime -= error
        print('Cluster Only: ',dprime)
        layer_info[label][layer]['pre'] = dprime   
             
            
        #After Lesion
       # print('Post-layer-lesioning')
        #print('Label:',label)
        #print('Layer:',layer)
        pred = clf.predict(X_new)
        loc = np.where(pred==label)[0]
        loc_new =[]
        for i in range(len(loc)):
            temp = np.where(pred_kmeans==loc[i])[0]
            temp2 = temp[np.asarray(np.where((temp >end) | (temp <=start))[0])]
            #print(len(temp), len(temp2))
            loc_new.extend(temp2)


        lambda_mask = np.ones(shape=((658272,)))
        lambda_mask[loc_new] = 0.
        print('post-loc', len(loc_new))
        sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
        model = AlexNet(weights_path="../../data/weights/alexnet_weights.h5",lambda_mask=lambda_mask)
        model.compile(optimizer=sgd, loss='mse')

        flag = 0
        dprime = 0.
        for p in classes:
            im_valid_test = []
            image_list_valid = '../../data/pkl/'+p+'_image_list_test.txt'
            with open(image_list_valid,'rb') as f:
                for line in f.readlines():
                    im_valid_test.append(line.strip('\n'))
            im_temp = preprocess_image(im_valid_test,227,227, color_mode="bgr")
            out = model.predict(im_temp,batch_size=64)

            true_valid_wids = []
            for i in im_valid_test:
                    temp1 = i.split('/')[4]
                    temp = temp1.split('.')[0].split('_')[2]
                    true_valid_wids.append(truth[int(temp)][1])

            predicted_valid_wids = []
            for i in range(len(im_valid_test)):
                #print im_list[i], pprint_output(out[i]), true_wids[i]
                predicted_valid_wids.append(pprint_output(out[i]))

            count, error  = top5accuracy(true_valid_wids, predicted_valid_wids)

            print str(p)+' '+str(count)+' '+str(len(im_valid_test))+' '+str(error)+' '+str(1-error)

            if flag == 0:
                dprime = error
                flag = 1
            else:
                dprime -= error
        print('Cluster - layer: ',dprime)
        layer_info[label][layer]['post'] = dprime
        
        
         #Random Lesion
       # print('Post-layer-lesioning')
        #print('Label:',label)
        #print('Layer:',layer)
        pred = clf.predict(X_new)
        loc = np.where(pred==label)[0]
        loc_new =[]
        for i in range(len(loc)):
            temp = np.where(pred_kmeans==loc[i])[0]
            temp2 = temp[np.asarray(np.where((temp >end) | (temp <=start))[0])]
            #print(len(temp), len(temp2))
            loc_new.extend(temp2)

        loc_new2 = np.random.randint(start,end,len(loc_new))
        lambda_mask = np.ones(shape=((658272,)))
        lambda_mask[loc_new2] = 0.
        print('post-rand-loc', len(loc_new))
        sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
        model = AlexNet(weights_path="../../data/weights/alexnet_weights.h5",lambda_mask=lambda_mask)
        model.compile(optimizer=sgd, loss='mse')

        flag = 0
        dprime = 0.
        for p in classes:
            im_valid_test = []
            image_list_valid = '../../data/pkl/'+p+'_image_list_test.txt'
            with open(image_list_valid,'rb') as f:
                for line in f.readlines():
                    im_valid_test.append(line.strip('\n'))
            im_temp = preprocess_image(im_valid_test,227,227, color_mode="bgr")
            out = model.predict(im_temp,batch_size=64)

            true_valid_wids = []
            for i in im_valid_test:
                    temp1 = i.split('/')[4]
                    temp = temp1.split('.')[0].split('_')[2]
                    true_valid_wids.append(truth[int(temp)][1])

            predicted_valid_wids = []
            for i in range(len(im_valid_test)):
                #print im_list[i], pprint_output(out[i]), true_wids[i]
                predicted_valid_wids.append(pprint_output(out[i]))

            count, error  = top5accuracy(true_valid_wids, predicted_valid_wids)

            print str(p)+' '+str(count)+' '+str(len(im_valid_test))+' '+str(error)+' '+str(1-error)

            if flag == 0:
                dprime = error
                flag = 1
            else:
                dprime -= error
        print('Random: ',dprime)
        layer_info[label][layer]['rand'] = dprime
       

In [ ]:
X = np.arange(1,8)
Y = []
Z =[]
tmp = 0.
for item in X:
    print(item,layer_info[ana][item]['pre'],layer_info[ana][item]['post'],layer_info[ana][item]['rand'])
    tmp += layer_info[ana][item]['pre']-layer_info[ana][item]['post']
    Y.append(layer_info[ana][item]['pre']-layer_info[ana][item]['post'])
    if layer_info[ana][item]['rand'] == 0.:
        Z.append(0.)
    else:
        Z.append(layer_info[ana][item]['rand']-layer_info[ana][item]['no'])
print tmp
width = 0.35  # the width of the bars

rects1 = plt.bar(X- width/2, Y, width, label='Layer-specific cluster lesions')
rects2 = plt.bar(X+width/2, Z, width, label='Random layer lesions')

plt.ylabel('Relative change in performance impact')
plt.xlabel('Different layers of Alexnet')
plt.xticks(X, ('conv_1', 'conv_2', 'conv_3', 'conv_4','conv_5','fc1','fc2'))
plt.title('Change in Performance by layer for Animate')
plt.legend()
plt.savefig('../../results/scree/animate_by_layer.png', format='png')
#plt.ylim([0,1])

In [ ]:
X = np.arange(1,8)
Y = []
Z = []
tmp = 0.
for item in X:
    print(item,layer_info[ina][item]['pre'],layer_info[ina][item]['post'],layer_info[ana][item]['rand'])
    Y.append((-1*layer_info[ina][item]['pre'])-(-1*layer_info[ina][item]['post']))
    if layer_info[ana][item]['rand'] == 0.:
        Z.append(0.)
    else:
        Z.append(layer_info[ana][item]['rand']-layer_info[ana][item]['no'])
    tmp += (-1*layer_info[ina][item]['pre'])-(-1*layer_info[ina][item]['post'])
print(tmp)
width = 0.35  # the width of the bars

rects1 = plt.bar(X- width/2, Y, width, label='Layer-specific cluster lesions')
rects2 = plt.bar(X+width/2, Z, width, label='Random layer lesions')

plt.ylabel('Relative change in performance impact')
plt.xticks(X, ('conv_1', 'conv_2', 'conv_3', 'conv_4','conv_5','fc1','fc2'))
plt.xlabel('Different layers of Alexnet')
plt.title('Change in Performance by layer for Inanimate')
plt.legend()
plt.savefig('../../results/scree/inaniamte_by_layer.png', format='png')
#plt.ylim([0,1])

In [22]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

def alexnet_model(img_shape=(227, 227, 3), n_classes=1000, l2_reg=0.,
	weights=None):

	# Initialize model
	alexnet = Sequential()

	# Layer 1
	alexnet.add(Conv2D(96, (11, 11), input_shape=img_shape,
		padding='same', kernel_regularizer=l2(l2_reg)))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 2
	alexnet.add(Conv2D(256, (5, 5), padding='same'))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 3
	alexnet.add(ZeroPadding2D((1, 1)))
	alexnet.add(Conv2D(512, (3, 3), padding='same'))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 4
	alexnet.add(ZeroPadding2D((1, 1)))
	alexnet.add(Conv2D(1024, (3, 3), padding='same'))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))

	# Layer 5
	alexnet.add(ZeroPadding2D((1, 1)))
	alexnet.add(Conv2D(1024, (3, 3), padding='same'))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 6
	alexnet.add(Flatten())
	alexnet.add(Dense(3072))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 7
	alexnet.add(Dense(4096))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 8
	alexnet.add(Dense(n_classes))
	alexnet.add(BatchNormalization())
	alexnet.add(Activation('softmax'))

	if weights is not None:
		alexnet.load_weights(weights)

	return alexnet

In [26]:
model = alexnet_model()

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 96, 227, 3)        2636928   
_________________________________________________________________
batch_normalization_17 (Batc (None, 96, 227, 3)        12        
_________________________________________________________________
activation_17 (Activation)   (None, 96, 227, 3)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 96, 113, 1)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 256, 113, 1)       614656    
_________________________________________________________________
batch_normalization_18 (Batc (None, 256, 113, 1)       4         
_________________________________________________________________
activation_18 (Activation)   (None, 256, 113, 1)       0         
__________

In [21]:
model1 = AlexNet(weights_path="../../data/weights/alexnet_weights.h5")
model1.summary()    

th
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 3, 227, 227)  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 96, 55, 55)   34944       input_2[0][0]                    
__________________________________________________________________________________________________
lambda_15 (Lambda)              (None, 96, 55, 55)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 96, 27, 27)   0           lambda_15[0][0]                  
__________________________________________________________________________________________________
convpoo

Total params: 60,965,224
Trainable params: 60,965,224
Non-trainable params: 0
__________________________________________________________________________________________________


In [ ]:
for layer in model.layers:
    weights = layer.get_weights()
    if len(weights) > 0:
        print layer.name,len(weights)

In [4]:
#LRP

import innvestigate
import innvestigate.utils
import imp
import keras.applications.vgg16 as vgg16

utils = imp.load_source("utils", 'utils.py')

# Get model
#model = AlexNet(weights_path="../../data/weights/alexnet_weights.h5")
model, preprocess = vgg16.VGG16(), vgg16.preprocess_input

# Strip softmax layer
model = innvestigate.utils.model_wo_softmax(model)

# Create analyzer
analyzer = innvestigate.create_analyzer("lrp.z", model,reverse_keep_tensors=True)


im_valid_test = []
image_list_valid = '../../data/pkl/'+p+'_image_list_test.txt'
with open(image_list_valid,'rb') as f:
    for line in f.readlines():
        im_valid_test.append(line.strip('\n'))
im_temp = preprocess_image(im_valid_test,227,227, color_mode="bgr")


image_activations = []
for image in im_temp:
    x = preprocess(image[None])
    # Apply analyzer w.r.t. maximum activated output-neuron
    a = analyzer.analyze(x)
    data = np.asarray(analyzer._reversed_tensors)
    activations = np.array([])
    for i  in range(1,len(model.layers)):
        j = i-1
        if model.layers[j].name.endswith('conv1') or model.layers[j].name.endswith('conv2') or model.layers[j].name.endswith('conv3') or  model.layers[j].name.endswith('fc1') or model.layers[j].name.endswith('fc2') :
            #print(model.layers[j].name,data[i][1].shape)
            temp = np.mean(data[i][1], axis=0).ravel()
            activations = np.append(activations,temp)
        image_activations.append(activations)

image_activations = np.asarray(image_activations)
image_activations = np.mean(image_activations, axis=0).ravel()
print(image_activations.shape)


number = 30 
index= sorted(range(len(image_activations), key=lambda k: abs(image_activations[k]),reverse=True))
print(index[0:number])

idx = np.ones(image_activations.shape)
idx[index[0:number]] = 0.

'''
def return_activations(model,image_list,algorithm='lrp.z'):
  analyzer = innvestigate.create_analyzer("lrp.z", model,reverse_keep_tensors=True)
  data = np.asarray(analyzer._reversed_tensors)
  data[22][1].shape

image_list = []
'''

NameError: name 'AlexNet' is not defined